This notebook is an implementation of a continuous Hopfield network for the purpose of remembering images.

It uses the continuous updating rule of $\tau_i \frac{dV_i}{dt} = -V_i + g_\beta (u_i) = -V_i + g_\beta (\sum_j W_{ij}V_j)$

Cheat sheet:

- $V$ is a vector of all the neurons. In our case, each neuron represents a pixel where the value is between $[-1, 1]$
- $g_\beta$ is a continuous non-linear function. In our case, we use tanh, which outputs values between $[-1, 1]$
- $W$ is the symmetric weight matrix representing the connections between neurons. $W_{ii} = 0$ and $W_{ij} = W_{ji}$. It is initialized by setting $W_{ij} = V_iV_j$ where $V$ is initialized to the state you want the network to "memorize". If you want the network to remember multiple of these states, simply set $W$ to the sum of all the $W$ matrices, one computed for each state.


In [2]:
import numpy as np

In [ ]:
num_images_to_generate = 3
image_width = 5
image_height = 3

num_neurons = image_width * image_height
images = []
# This is just an example to get it working, but "images" will be an array of matrices, where each matrix is a different image
for i in range(num_images_to_generate):
    images.append(np.random.uniform(-1.0, 1.0, size=(image_width, image_height)))

W = np.zeros((num_neurons, num_neurons))

# computes W_{ij} = V_i * V_j for all i and j. (Saves all images in weight matrix)
for image in images:
    W = W + np.outer(image, image)


In [ ]:
from scipy.integrate import solve_ivp

# constants
def g(u, b):
    return np.tanh(u * b)

tol = 1e-6 # integrate until converging to this tolerance
tau = 1.0
dt = 0.1
beta = 1.0

def hopfield_diff_eq(t, v):
    u = np.dot(W, v) # check if this is equivalent to sum(W_ij * V_j) for all j
    dvdt = (-v + g(u, beta))/tau
    return dvdt

V = np.random.uniform(-1.0, 1.0, size=(num_neurons))


time = 0 # for integration, this is the starting time at first
times = [time]
states = [V]

current_state = V
prev_state = V + 2 * tol # This makes the first iteration of the loop work

while np.abs(current_state - prev_state).max() > tol:
    prev_state = current_state
    result = solve_ivp(hopfield_diff_eq, (time, time + dt), current_state, method='RK45', t_eval=[time+dt])
    time += dt
    current_state = result.y[:, -1]

    times.append(time)
    states.append(current_state)

times = np.array(times)
states = np.array(states)
